In [ ]:
#!/usr/bin/env python3
"""
LPU Class Notification Bot - Complete Version with AddTimetable Command
Based on your actual timetable PDF (VID: 12313773)
"""

import asyncio
import logging
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional

# Try to import nest_asyncio for Jupyter compatibility
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.WARNING
)
logger = logging.getLogger(__name__)

# ==================== CONFIGURATION ====================
BOT_TOKEN = "8325917697:AAEbz3_pmzTjbZQ17Nc53c1tPurwK2Cb384"
CLASSES_FILE = "lpu_classes.json"

# LPU Course mappings for better display
COURSE_INFO = {
    "CSE322": {"name": "Formal Languages & Automation Theory", "faculty": "Priyanka Gotter", "room": "MyClass-1"},
    "PETS13": {"name": "Data Structure II", "faculty": "Alok Kumar", "room": "MyClass-1"},
    "PEA306": {"name": "Analytical Skills II", "faculty": "Abhishek Raj", "room": "MyClass-1"},
    "CSE343": {"name": "Training in Programming", "faculty": "TBD", "room": "Lab"},
    "FIN214": {"name": "Intro to Financial Markets", "faculty": "TBD", "room": "MyClass-1"},
    "INT234": {"name": "Predictive Analytics", "faculty": "TBD", "room": "MyClass-1"},
    "INT374": {"name": "Data Analytics with Power BI", "faculty": "TBD", "room": "MyClass-1"},
    "PEV301": {"name": "Verbal Ability", "faculty": "TBD", "room": "MyClass-1"}
}

class LPUClassBot:
    def __init__(self):
        self.classes = self.load_classes()
        self.application = None
        self.running = False
        self.reminder_sent = set()
    
    def load_classes(self) -> Dict:
        """Load classes from JSON file with error handling"""
        try:
            if os.path.exists(CLASSES_FILE):
                with open(CLASSES_FILE, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    if isinstance(data, dict):
                        return data
                    return {}
            return {}
        except (json.JSONDecodeError, FileNotFoundError) as e:
            logger.error(f"Error loading classes: {e}")
            return {}
    
    def save_classes(self):
        """Save classes to JSON file with backup"""
        try:
            backup_file = f"{CLASSES_FILE}.backup"
            
            if os.path.exists(backup_file):
                os.remove(backup_file)
            
            if os.path.exists(CLASSES_FILE):
                os.rename(CLASSES_FILE, backup_file)
            
            with open(CLASSES_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.classes, f, indent=2, ensure_ascii=False)
            
            if os.path.exists(backup_file):
                os.remove(backup_file)
                
        except Exception as e:
            logger.error(f"Error saving classes: {e}")
            backup_file = f"{CLASSES_FILE}.backup"
            if os.path.exists(backup_file) and not os.path.exists(CLASSES_FILE):
                try:
                    os.rename(backup_file, CLASSES_FILE)
                except Exception as restore_error:
                    logger.error(f"Error restoring backup: {restore_error}")
    
    def parse_class_input(self, input_text: str) -> Optional[Dict]:
        """Parse class input with improved validation"""
        parts = [part.strip() for part in input_text.split('|')]
        
        if len(parts) < 2:
            return None
        
        try:
            class_name = parts[0].strip()
            class_time_str = parts[1].strip()
            reminder_minutes = int(parts[2].strip()) if len(parts) > 2 and parts[2].strip().isdigit() else 15
            class_url = parts[3].strip() if len(parts) > 3 else ""
            notes = parts[4].strip() if len(parts) > 4 else ""
            
            time_formats = [
                "%Y-%m-%d %H:%M",
                "%Y-%m-%d %H:%M:%S",
                "%d/%m/%Y %H:%M",
                "%d-%m-%Y %H:%M"
            ]
            
            class_time = None
            for fmt in time_formats:
                try:
                    class_time = datetime.strptime(class_time_str, fmt)
                    break
                except ValueError:
                    continue
            
            if not class_time:
                return None
            
            if reminder_minutes < 1 or reminder_minutes > 120:
                reminder_minutes = 15
            
            return {
                "name": class_name,
                "time": class_time.isoformat(),
                "reminder_minutes": reminder_minutes,
                "url": class_url,
                "notes": notes
            }
            
        except Exception as e:
            logger.error(f"Parse error: {e}")
            return None
    
    def add_class(self, user_id: int, class_data: Dict) -> int:
        """Add a new class with validation"""
        user_key = str(user_id)
        if user_key not in self.classes:
            self.classes[user_key] = []
        
        existing_ids = [cls.get('id', 0) for cls in self.classes[user_key]]
        new_id = max(existing_ids, default=0) + 1
        
        class_entry = {
            "id": new_id,
            "name": class_data["name"],
            "time": class_data["time"],
            "reminder_minutes": class_data["reminder_minutes"],
            "url": class_data["url"],
            "notes": class_data["notes"],
            "reminded": False,
            "started": False,
            "created_at": datetime.now().isoformat()
        }
        
        self.classes[user_key].append(class_entry)
        self.save_classes()
        return new_id
    
    def get_user_classes(self, user_id: int) -> List:
        """Get all classes for a user, sorted by time"""
        user_classes = self.classes.get(str(user_id), [])
        return sorted(user_classes, key=lambda x: x.get('time', ''))
    
    def remove_class(self, user_id: int, class_id: int) -> bool:
        """Remove a class by ID"""
        user_classes = self.classes.get(str(user_id), [])
        for i, cls in enumerate(user_classes):
            if cls.get("id") == class_id:
                user_classes.pop(i)
                self.save_classes()
                return True
        return False
    
    def get_upcoming_classes(self, user_id: int, limit: int = 5) -> List:
        """Get upcoming classes for a user"""
        user_classes = self.get_user_classes(user_id)
        now = datetime.now()
        upcoming = []
        
        for cls in user_classes:
            try:
                class_time = datetime.fromisoformat(cls["time"])
                if class_time > now:
                    upcoming.append(cls)
            except (ValueError, KeyError):
                continue
        
        return sorted(upcoming, key=lambda x: x["time"])[:limit]
    
    def get_course_info(self, class_name: str) -> Dict:
        """Get course information based on class name"""
        for code, info in COURSE_INFO.items():
            if code in class_name.upper():
                return info
        return {"name": class_name, "faculty": "TBD", "room": "TBD"}
    
    def cleanup_old_classes(self, days_old: int = 7):
        """Remove classes older than specified days"""
        cutoff_date = datetime.now() - timedelta(days=days_old)
        
        for user_id, user_classes in self.classes.items():
            self.classes[user_id] = [
                cls for cls in user_classes
                if datetime.fromisoformat(cls["time"]) > cutoff_date
            ]
        
        self.save_classes()
    
    async def check_reminders(self):
        """Check for upcoming classes and send reminders"""
        while self.running:
            try:
                now = datetime.now()
                for user_id, user_classes in self.classes.items():
                    for cls in user_classes:
                        if cls.get('reminded', False):
                            continue
                        
                        try:
                            class_time = datetime.fromisoformat(cls["time"])
                            reminder_time = class_time - timedelta(minutes=cls["reminder_minutes"])
                            
                            if now >= reminder_time and not cls.get('reminded'):
                                await self.send_reminder(int(user_id), cls)
                                cls['reminded'] = True
                                self.save_classes()
                        except Exception as e:
                            logger.error(f"Reminder error: {e}")
                            
                await asyncio.sleep(60)
            except Exception as e:
                logger.error(f"Reminder loop error: {e}")
                await asyncio.sleep(60)
    
    async def send_reminder(self, user_id: int, class_data: Dict):
        """Send reminder message to user"""
        try:
            class_time = datetime.fromisoformat(class_data["time"])
            course_info = self.get_course_info(class_data["name"])
            
            reminder_msg = f"""
🔔 *Class Reminder!*

📚 **{class_data['name']}**
📖 {course_info['name']}
👨‍🏫 {course_info['faculty']}
📅 {class_time.strftime('%A, %B %d')}
⏰ Starts in {class_data['reminder_minutes']} minutes ({class_time.strftime('%I:%M %p')})

{f"🔗 [Join Class]({class_data['url']})" if class_data.get('url') else ""}
{f"📝 {class_data['notes']}" if class_data.get('notes') else ""}

Get ready! 🚀📖
            """
            
            keyboard = []
            if class_data.get('url'):
                keyboard.append([InlineKeyboardButton("🔗 Join Now", url=class_data['url'])])
            keyboard.append([InlineKeyboardButton("⏰ Next Classes", callback_data="list_classes")])
            
            reply_markup = InlineKeyboardMarkup(keyboard) if keyboard else None
            
            if self.application:
                await self.application.bot.send_message(
                    chat_id=user_id,
                    text=reminder_msg,
                    parse_mode='Markdown',
                    disable_web_page_preview=True,
                    reply_markup=reply_markup
                )
                
        except Exception as e:
            logger.error(f"Send reminder error: {e}")

# Create global bot instance
bot = LPUClassBot()

# ==================== COMMAND HANDLERS ====================

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Enhanced start command with LPU branding"""
    user_name = update.effective_user.first_name or "Student"
    
    welcome_text = f"""
🎓 *Welcome to LPU Class Bot, {user_name}!*

Never miss your Code Tantra classes again! Perfect for your LPU timetable.

*🚀 Quick Commands:*
📚 `/add` - Add new class
📋 `/list` - View all classes  
⏰ `/next` - Next class
🗑️ `/remove ID` - Delete class
📊 `/today` - Today's classes
🗓️ `/addtimetable` - Add complete weekly schedule
❓ `/help` - Full help

*📖 Quick Add Example:*
`/add CSE322 FLAT | 2025-09-15 09:00 | 15 | https://myclass.lpu.in/cse322 | Formal Languages - Priyanka Gotter`

*🎯 Complete Timetable:*
Use `/addtimetable week` to add your entire weekly schedule automatically!

Ready to organize your semester! 🌟
    """
    
    keyboard = [
        [
            InlineKeyboardButton("📚 Add Class", callback_data="help_add"),
            InlineKeyboardButton("📋 My Classes", callback_data="list_classes")
        ],
        [
            InlineKeyboardButton("🗓️ Add Timetable", callback_data="help_addtimetable"),
            InlineKeyboardButton("⏰ Next Class", callback_data="next_class")
        ],
        [
            InlineKeyboardButton("❓ Help", callback_data="show_help")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    
    await update.message.reply_text(
        welcome_text, 
        parse_mode='Markdown', 
        reply_markup=reply_markup
    )

async def addtimetable_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Add complete LPU timetable based on your PDF"""
    
    if not context.args:
        timetable_msg = """
🗓️ *Add Complete LPU Timetable*
*Based on your actual PDF timetable (VID: 12313773)*

*Usage Options:*
• `/addtimetable week` - Add current week's schedule
• `/addtimetable next` - Add next week's schedule  
• `/addtimetable custom YYYY-MM-DD` - Add from specific date

*📚 Your LPU Schedule Pattern (Mon-Fri):*

**Every Day:**
• 09:00-10:00 AM - CSE322 FLAT (Priyanka Gotter)
• 10:00-11:00 AM - CSE322 FLAT (Priyanka Gotter)
• 01:00-02:00 PM - PEA306 Analytics (Abhishek Raj)
• 02:00-03:00 PM - PEA306 Analytics (Abhishek Raj)

**Monday & Tuesday:**
• 11:00-12:00 PM - PETS13 DS-II (Alok Kumar)
• 04:00-05:00 PM - PETS13 DS-II (Alok Kumar)

**Wednesday, Thursday, Friday:**
• 11:00-12:00 PM - PETS13 DS-II (Alok Kumar)
• 04:00-05:00 PM - Project Work (No class)

**Total Classes:** 27 per week
**Room:** MyClass-1 for all classes
        """
        
        keyboard = [
            [
                InlineKeyboardButton("📅 This Week", callback_data="timetable_week"),
                InlineKeyboardButton("📅 Next Week", callback_data="timetable_next")
            ],
            [
                InlineKeyboardButton("🎯 Custom Date", callback_data="timetable_custom"),
                InlineKeyboardButton("❌ Cancel", callback_data="cancel_timetable")
            ]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        
        await update.message.reply_text(
            timetable_msg,
            parse_mode='Markdown',
            reply_markup=reply_markup
        )
        return
    
    # Parse arguments
    option = context.args[0].lower()
    now = datetime.now()
    
    if option == "week":
        start_date = now - timedelta(days=now.weekday())
    elif option == "next":
        start_date = now - timedelta(days=now.weekday()) + timedelta(weeks=1)
    elif option == "custom" and len(context.args) > 1:
        try:
            start_date = datetime.strptime(context.args[1], '%Y-%m-%d')
        except ValueError:
            await update.message.reply_text(
                "❌ Invalid date format! Use: YYYY-MM-DD\n"
                "Example: `/addtimetable custom 2025-09-15`"
            )
            return
    else:
        await update.message.reply_text(
            "❌ Invalid option!\n"
            "Use: `/addtimetable week`, `/addtimetable next`, or `/addtimetable custom YYYY-MM-DD`"
        )
        return
    
    # Your exact LPU timetable pattern based on PDF
    added_count = 0
    failed_count = 0
    
    # Add classes for 5 days (Monday to Friday)
    for day_offset in range(5):
        class_date = start_date + timedelta(days=day_offset)
        day_name = class_date.strftime('%A')
        
        # Common classes for all weekdays
        daily_classes = [
            ("CSE322 FLAT", 9, 0, 15, "https://myclass.lpu.in/cse322", f"Formal Languages - Priyanka Gotter - {day_name}"),
            ("CSE322 FLAT", 10, 0, 10, "https://myclass.lpu.in/cse322", f"FLAT Class 2 - Priyanka Gotter - {day_name}"),
            ("PEA306 Analytics", 13, 0, 15, "https://myclass.lpu.in/pea306", f"Analytical Skills-II - Abhishek Raj - {day_name}"),
            ("PEA306 Analytics", 14, 0, 10, "https://myclass.lpu.in/pea306", f"Analytics Class 2 - Abhishek Raj - {day_name}")
        ]
        
        # PETS13 classes - varies by day according to your PDF
        if day_name in ['Monday', 'Tuesday']:
            # Mon/Tue: Full PETS13 schedule (11 AM + 4 PM)
            daily_classes.extend([
                ("PETS13 DS-II", 11, 0, 15, "https://myclass.lpu.in/pets13", f"Data Structure-II - Alok Kumar - {day_name}"),
                ("PETS13 DS-II", 16, 0, 15, "https://myclass.lpu.in/pets13", f"DS Evening - Alok Kumar - {day_name}")
            ])
        elif day_name in ['Wednesday', 'Thursday', 'Friday']:
            # Wed/Thu/Fri: Only morning PETS13 (4 PM is Project Work)
            daily_classes.append(
                ("PETS13 DS-II", 11, 0, 15, "https://myclass.lpu.in/pets13", f"Data Structure-II - Alok Kumar - {day_name}")
            )
        
        # Add all classes for this day
        for class_name, hour, minute, reminder_min, url, notes in daily_classes:
            class_time = class_date.replace(hour=hour, minute=minute, second=0, microsecond=0)
            
            class_data = {
                "name": class_name,
                "time": class_time.isoformat(),
                "reminder_minutes": reminder_min,
                "url": url,
                "notes": notes
            }
            
            try:
                bot.add_class(update.effective_user.id, class_data)
                added_count += 1
            except Exception as e:
                logger.error(f"Failed to add class: {e}")
                failed_count += 1
    
    # Results message
    success_msg = f"""
✅ *LPU Timetable Added Successfully!*

📊 **Results:**
✅ Added: {added_count} classes
❌ Failed: {failed_count} classes

📅 **Schedule Period:**
From: {start_date.strftime('%A, %B %d, %Y')}
To: {(start_date + timedelta(days=4)).strftime('%A, %B %d, %Y')}

📚 **Your LPU Classes Added:**
• CSE322 FLAT - 10 sessions (Priyanka Gotter)
• PETS13 DS-II - 8 sessions (Alok Kumar)
• PEA306 Analytics - 10 sessions (Abhishek Raj)

⏰ **Schedule Pattern:**
• Mon/Tue: 6 classes each day
• Wed/Thu/Fri: 5 classes each day (Project Work at 4 PM)

🔔 **Reminders:** All set up automatically
🌐 **MyClass Links:** Ready for quick joining

Use `/list` to view your complete schedule!
    """
    
    keyboard = [
        [
            InlineKeyboardButton("📋 View Schedule", callback_data="list_classes"),
            InlineKeyboardButton("⏰ Next Class", callback_data="next_class")
        ],
        [
            InlineKeyboardButton("📊 Today's Classes", callback_data="today_classes"),
            InlineKeyboardButton("📅 This Week", callback_data="week_classes")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    
    await update.message.reply_text(
        success_msg,
        parse_mode='Markdown',
        reply_markup=reply_markup
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Comprehensive help command"""
    help_text = """
🤖 *LPU Class Bot - Complete Guide*

*🗓️ Quick Timetable Setup:*
`/addtimetable week` - Add entire week's schedule (27 classes)
`/addtimetable next` - Add next week's schedule
`/addtimetable custom 2025-09-15` - From specific date

*📚 Manual Class Adding:*
`/add Class Name | YYYY-MM-DD HH:MM | Minutes | URL | Notes`

*🎯 Your LPU Course Examples:*
```
/add CSE322 FLAT | 2025-09-15 09:00 | 15 | https://myclass.lpu.in/cse322 | Formal Languages - Priyanka Gotter

/add PETS13 DS-II | 2025-09-15 11:00 | 15 | https://myclass.lpu.in/pets13 | Data Structure - Alok Kumar

/add PEA306 Analytics | 2025-09-15 13:00 | 20 | https://myclass.lpu.in/pea306 | Analytical Skills - Abhishek Raj
```

*📋 Management Commands:*
• `/list` - Show all classes
• `/next` - Next upcoming class  
• `/today` - Today's schedule
• `/week` - This week's classes
• `/remove 1` - Delete class ID 1
• `/clear` - Remove all classes
• `/status` - Bot statistics
• `/test` - Test bot functionality

*💡 Pro Tips:*
• Bot sends automatic reminders
• Classes auto-save to file
• Use `/addtimetable week` for instant setup
• All classes include MyClass LPU links
• Respects your Project Work schedule

Need help? Just ask! 🎓
    """
    await update.message.reply_text(help_text, parse_mode='Markdown')

async def add_class_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Enhanced add class command"""
    if not context.args:
        example_text = """
📚 *Add Your LPU Class*

*Format:*
`/add Class Name | YYYY-MM-DD HH:MM | Minutes | URL | Notes`

*🎯 Your Course Examples:*
```
/add CSE322 FLAT | 2025-09-15 09:00 | 15 | https://myclass.lpu.in/cse322 | Formal Languages - Priyanka Gotter

/add PETS13 DS-II | 2025-09-15 11:00 | 15 | https://myclass.lpu.in/pets13 | Data Structure - Alok Kumar

/add PEA306 Analytics | 2025-09-15 13:00 | 20 | https://myclass.lpu.in/pea306 | Analytical Skills - Abhishek Raj
```

*🗓️ Quick Option:*
Use `/addtimetable week` to add your complete weekly schedule automatically!
        """
        await update.message.reply_text(example_text, parse_mode='Markdown')
        return
    
    try:
        input_text = ' '.join(context.args)
        parsed_data = bot.parse_class_input(input_text)
        
        if not parsed_data:
            await update.message.reply_text(
                "❌ *Invalid format!*\n\n"
                "Use: `/add Class Name | YYYY-MM-DD HH:MM | Minutes | URL | Notes`\n"
                "Example: `/add CSE322 FLAT | 2025-09-15 09:00 | 15 | https://myclass.lpu.in/cse322 | Formal Languages - Priyanka Gotter`",
                parse_mode='Markdown'
            )
            return
        
        if "error" in parsed_data:
            await update.message.reply_text(f"❌ {parsed_data['error']}")
            return
        
        class_id = bot.add_class(update.effective_user.id, parsed_data)
        course_info = bot.get_course_info(parsed_data["name"])
        class_time = datetime.fromisoformat(parsed_data["time"])
        
        success_msg = f"""
✅ *Class Added Successfully!*

📚 **{parsed_data['name']}**
📖 {course_info['name']}
👨‍🏫 {course_info['faculty']}
📅 {class_time.strftime('%A, %B %d')}
⏰ {class_time.strftime('%I:%M %p')}
🔔 {parsed_data['reminder_minutes']}-min reminder
🆔 ID: {class_id}
{f"🔗 [Join Class]({parsed_data['url']})" if parsed_data['url'] else ""}
{f"📝 {parsed_data['notes']}" if parsed_data['notes'] else ""}

I'll remind you when it's time! ⏰✨
        """
        
        keyboard = [
            [InlineKeyboardButton("📋 View All Classes", callback_data="list_classes")],
            [InlineKeyboardButton("➕ Add Another", callback_data="help_add")]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        
        await update.message.reply_text(
            success_msg, 
            parse_mode='Markdown',
            disable_web_page_preview=True,
            reply_markup=reply_markup
        )
        
    except Exception as e:
        logger.error(f"Error adding class: {e}")
        await update.message.reply_text(
            "❌ Something went wrong! Please check your format and try again.\n\n"
            "Use `/help` for examples! 📚"
        )

async def list_classes_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Enhanced list command"""
    user_classes = bot.get_user_classes(update.effective_user.id)
    
    if not user_classes:
        await update.message.reply_text(
            "📚 *No classes scheduled yet!*\n\n"
            "Ready to add your LPU timetable?\n\n"
            "🚀 **Quick Setup:**\n"
            "`/addtimetable week` - Add complete weekly schedule\n\n"
            "📝 **Manual Add:**\n"
            "`/add CSE322 FLAT | 2025-09-15 09:00 | 15 | URL | Notes`",
            parse_mode='Markdown'
        )
        return
    
    now = datetime.now()
    upcoming_classes = []
    past_classes = []
    
    for cls in user_classes:
        try:
            class_time = datetime.fromisoformat(cls["time"])
            if class_time > now:
                upcoming_classes.append(cls)
            else:
                past_classes.append(cls)
        except (ValueError, KeyError):
            continue
    
    message = f"📋 *Your LPU Classes ({len(user_classes)} total)*\n\n"
    
    if upcoming_classes:
        message += "⏰ *UPCOMING CLASSES:*\n"
        for cls in upcoming_classes[:10]:
            try:
                class_time = datetime.fromisoformat(cls["time"])
                course_info = bot.get_course_info(cls["name"])
                time_diff = class_time - now
                
                if time_diff.days > 0:
                    time_until = f"in {time_diff.days} day(s)"
                elif time_diff.seconds > 3600:
                    hours = time_diff.seconds // 3600
                    time_until = f"in {hours}h"
                else:
                    minutes = time_diff.seconds // 60
                    time_until = f"in {minutes}min"
                
                message += f"""
🎯 **ID {cls['id']}** - {cls['name']}
📖 {course_info['name']}
👨‍🏫 {course_info['faculty']}
📅 {class_time.strftime('%a %b %d, %I:%M %p')}
⏱️ {time_until} • 🔔 {cls['reminder_minutes']}min
{f"🔗 [Link]({cls['url']})" if cls.get('url') else ""}
{f"📝 {cls['notes']}" if cls.get('notes') else ""}
━━━━━━━━━━━━━━━━
                """
            except (ValueError, KeyError):
                continue
    
    if past_classes:
        message += f"\n📜 *RECENT PAST CLASSES ({len(past_classes)}):*\n"
        for cls in past_classes[-3:]:
            try:
                class_time = datetime.fromisoformat(cls["time"])
                message += f"✅ **{cls['name']}** - {class_time.strftime('%a %b %d')}\n"